In [2]:
import duckdb
from minio import Minio
con=duckdb.connect("play.db")
con.sql("install 'httpfs';")
con.sql("load 'httpfs';")
con.sql("SET s3_url_style='path';")
con.sql(" SET s3_endpoint='localhost:8050'")
con.sql("SET s3_use_ssl=false;")
con.sql("SET s3_access_key_id='devkey';")
con.sql("SET s3_secret_access_key='devpassword';")
# con.sql(" select * from read_parquet('s3://auctus-bucket/df.parquet');")

In [4]:
try:
    dataframe=con.sql("describe select * from read_parquet('s3://auctus-bucket/df.parquet') desc")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: Parser Error: syntax error at or near "desc"
LINE 1: ...rquet('s3://auctus-bucket/df.parquet') desc
                                                  ^


In [31]:
client = Minio(
        "localhost:8050",
        access_key="devkey",
        secret_key="devpassword",
        secure=False
    )

found = client.bucket_exists("auctus-bucket")
if not found:
    client.make_bucket("auctus-bucket")
else:
    print("Bucket 'auctus-bucket' already exists")



Bucket 'auctus-bucket' already exists


In [32]:
try:
    response = client.get_object("auctus-bucket", "dfxx.parquet")
except:
    print("file not found")


file not found


In [33]:
from io import BytesIO
parquet_buffer = BytesIO()
dataframe.to_parquet(parquet_buffer)

# Upload the Parquet file to MinIO
parquet_buffer.seek(0)
client.put_object(
    bucket_name="auctus-bucket",
    object_name="my-data.parquet",
    data=parquet_buffer,
    length=parquet_buffer.getbuffer().nbytes,
    content_type="application/octet-stream"
)